In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

In [2]:
sentiment = pd.read_csv('D:/jupyter/DATA_SCIENRE/Tuan5/test.csv', encoding='latin-1')
sentiment.sample(5)

,id,sentiment,review
16891,3348_7,1,Clint Tollinger arrives in a small town lookin...
19858,963_4,0,'Chances Are' a big mistake to see. You could ...
22894,5703_7,1,I'm a fan of Crash and Blade Runner and this m...
12775,5792_1,0,Another made for TV piece of junk! This is an ...
16044,2857_1,0,I've seen plenty of Sci-Fi Channel Original mo...


In [3]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [4]:
twits = [
    'This is amazing!',
    'ML is the best, yes it is',
    'I am not sure about how this is going to end...'
]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# Define an object of CountVectorizer() fit and transfom your twits into a 'bag'
count = CountVectorizer()
bag = count.fit_transform(twits)

In [6]:
# Find in document of CountVectorizer a function that show us list of feature names
count.get_feature_names()

['about',
 'am',
 'amazing',
 'best',
 'end',
 'going',
 'how',
 'is',
 'it',
 'ml',
 'not',
 'sure',
 'the',
 'this',
 'to',
 'yes']

In [7]:
# Call toarray() on your 'bag' to see the feature vectors
bag.toarray()

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]], dtype=int64)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
# Feed the tf-idf Vectorizer with twits using fit_transform()
tfidf_vec = tfidf.fit_transform(twits)

# Formatting the number to 2 digits after the decimal point by showing on this notebook
np.set_printoptions(precision=2)
# To print array in one line
np.set_printoptions(linewidth=np.inf)
print(tfidf.get_feature_names())
print(tfidf_vec.toarray())

['about', 'am', 'amazing', 'best', 'end', 'going', 'how', 'is', 'it', 'ml', 'not', 'sure', 'the', 'this', 'to', 'yes']
[[0.   0.   0.72 0.   0.   0.   0.   0.43 0.   0.   0.   0.   0.   0.55 0.   0.  ]
 [0.   0.   0.   0.4  0.   0.   0.   0.47 0.4  0.4  0.   0.   0.4  0.   0.   0.4 ]
 [0.33 0.33 0.   0.   0.33 0.33 0.33 0.2  0.   0.   0.33 0.33 0.   0.25 0.33 0.  ]]


In [10]:
from collections import Counter

vocab = Counter()
for twit in sentiment.review:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

[('the', 286666),
 ('a', 154921),
 ('and', 152512),
 ('of', 142817),
 ('to', 132392),
 ('is', 103103),
 ('in', 85484),
 ('I', 65901),
 ('that', 64475),
 ('this', 57131),
 ('it', 54369),
 ('/><br', 50870),
 ('was', 46655),
 ('as', 42466),
 ('with', 41670),
 ('for', 41031),
 ('but', 33733),
 ('The', 33713),
 ('on', 30730),
 ('movie', 30420)]

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [12]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

vocab_reduced = Counter()
# Go through all of the items of vocab using vocab.items() and pick only words that are not in 'stop_words' 
# and save them in vocab_reduced
for w, c in vocab.items():
    if not w in stop_words:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

[('I', 65901),
 ('/><br', 50870),
 ('The', 33713),
 ('movie', 30420),
 ('film', 27385),
 ('one', 20660),
 ('like', 18110),
 ('This', 12272),
 ('would', 11908),
 ('good', 11421),
 ('It', 10943),
 ('really', 10795),
 ('even', 10594),
 ('see', 10145),
 ('-', 9347),
 ('get', 8762),
 ('story', 8518),
 ('much', 8493),
 ('time', 7747),
 ('make', 7472)]

In [13]:
import re 

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

# Create some random texts for testing the function preprocessor()
print(preprocessor('I like it :), |||<><>'))

i like it  :)


In [14]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

# Split a text into list of words
def tokenizer(text):
    return text.split()

# Split a text into list of words and apply stemming technic
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# Testing
print(tokenizer('Hi there, I am loving this, like with a lot of love'))
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

['Hi', 'there,', 'I', 'am', 'loving', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']
['Hi', 'there,', 'I', 'am', 'love', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']


In [16]:
from sklearn.model_selection import train_test_split

X = sentiment['review']
y = sentiment['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

# A pipeline is what chains several steps together, once the initial exploration is done. 
# For example, some codes are meant to transform features — normalise numericals, or turn text into vectors, 
# or fill up missing data, they are transformers; other codes are meant to predict variables by fitting an algorithm,
# they are estimators. Pipeline chains all these together which can then be applied to training data
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.885
confusion matrix:
 [[2165  338]
 [ 237 2260]]
classification report:
              precision    recall  f1-score   support

          0       0.90      0.86      0.88      2503
          1       0.87      0.91      0.89      2497

avg / total       0.89      0.89      0.88      5000



In [20]:
twits = [
    "I do not feel not bad", # Phuc +1
    'This model is "so good" :))', # Long -1
    'we are who we are', # Nghi 0
    'its good to be bad sometimes', # PA +1
    'what a wonderful failure! (sarcasm :)))', #Phuc +1
    'People do not like the bad things', # Chi 0
    'We finally have the test result. You are positive', # Long +1
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

I do not feel not bad --> Negative, Positive = [0.98 0.02]
This model is "so good" :)) --> Negative, Positive = [0.25 0.75]
we are who we are --> Negative, Positive = [0.49 0.51]
its good to be bad sometimes --> Negative, Positive = [0.77 0.23]
what a wonderful failure! (sarcasm :))) --> Negative, Positive = [0.3 0.7]
People do not like the bad things --> Negative, Positive = [0.97 0.03]
We finally have the test result. You are positive --> Negative, Positive = [0.7 0.3]


In [21]:
import pickle
import os
pickle.dump(clf, open('logisticRegression.pkl', 'wb'))

In [22]:
with open('logisticRegression.pkl', 'rb') as model:
    reload_model = pickle.load(model)
preds = reload_model.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

I do not feel not bad --> Negative, Positive = [0.98 0.02]
This model is "so good" :)) --> Negative, Positive = [0.25 0.75]
we are who we are --> Negative, Positive = [0.49 0.51]
its good to be bad sometimes --> Negative, Positive = [0.77 0.23]
what a wonderful failure! (sarcasm :))) --> Negative, Positive = [0.3 0.7]
People do not like the bad things --> Negative, Positive = [0.97 0.03]
We finally have the test result. You are positive --> Negative, Positive = [0.7 0.3]


In [33]:
from sklearn.model_selection import train_test_split

X = sentiment[['id', 'sentiment', 'review']].values
y = sentiment[['id', 'sentiment', 'review']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

print('Training set:', X_train.shape, y_train.shape)
print('Test set:', X_test.shape, y_test.shape)

Training set: (20000, 3) (20000, 3)
Test set: (5000, 3) (5000, 3)


In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import log_loss

# Create Logistics Regression model from X and y
lg = LogisticRegression()
lg.fit(X_train, y_train)
predictions = lg.predict(X_test)

# Show metrics
print("Accuracy score: %f" % accuracy_score(y_test, predictions))
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print('Log loss:', log_loss(y_test, predictions)/len(y_test))

ValueError: could not convert string to float: 'Another attempt by modern Japanese directors to redefine the chambara genre. Successful, and not, in varying degrees.<br /><br />Buddhist monk has a vision that he is to slay a legendary (and very active) demon at the Gojoe bridge in order to attain enlightenment. While not at the forefront, Buddhist thought is at the heart of this movie, much like Kurosawa\'s \\Ran\\". It probably what made the movie the most interesting to me although it\'s nowheres near \\"Ran\'s\\" league. <br /><br />Stylish visual direction and excellent photography keep the movie mostly interesting throughout the two hour and eighteen minute length. The lead actors are uniformly excellent. The music is really good. The two weaknesses are the story and the fight scenes. The movie drags in the middle which could have been fixed by some prudent editing. And the fight scenes are mostly filmed in blurry close-ups. This works for most of the film but the finale feels lik